
# Trabajo Práctico 2


Descargar el dataset **Facebook comment volume** https://archive.ics.uci.edu/ml/datasets/Facebook+Comment+Volume+Dataset. 

El dataset tiene diferentes versiones, utilizaremos la **número 5** para train y para test **Features_TestSet**. 

Notar que trae **carpeta de train y de test**. 

Una descripcióm completa del dataset pueden encontrarla en el link. Se trata de publicaciones de Facebook y se quiere **predecir la columna 54: cantidad de comentarios en las próximas H horas"**.

El dataset contiene variables en muchos formatos diferentes, algunos de los cuales no hemos trabajado o no lo hemos hechos en profundidad. Esas variables pueden ser descartadas o bien hacer una investigación personal para ver de qué manera poder incluirlas para mejorar los resultados obtenidos.



## Notas santi

The task here is to estimate the comment count that a post is expected to receive in next few hours.
La unidad basica es el post. Total posts: 51108. De los que quedan 80% en train (40998). y 20% en test (10120).

Target value: Comentarios que recibe el post. 
NOTE: Given some posts that appeared in past, whose target values (comments received) are already known, we simulated the scenario.

Los autores lo consideran un problema de regresion (porque se estima un valor numerico que es el comment count)

### Respecto al tiempo. 

Base date/time. El autor establece una hora y una fecha como BD/T arbitrariamente. Y luego toma los comentarios que ocurrieron los ultimos tres dias. Filtra un par. Y luego se queda con un total de posts de 51108. Este es el data set completo antes del split. 

Luego el split de train y testing se hace en base al tiempo. Se selecciona arbitrariamente una fecha/hora y lo que esta antes es de train (el pasado), y lo que esta despues es de test(el futuro)


It is selected to simulate the scenario, as we already know what will happen after this. There is one more kind of time we used in this formulation: is the post published time, which comes before the selected base date/time.

Primero parte el D-S en dos definiendo una linea de corte temporal arbitraria. Esto hace que el set se divida en 80 por ciento de post mas veijos que la linea de corte, y 20 porciento mas nuevos que la linea de corte. 

Una vez dividido en test y train, establece *variantes* para cada uno. el proceso de gerneracion de variantes en distinto en train y en test. 





### Features
53 en total + el target value para cada **post** 

- 4 features meta de la pagina en donde esta el post. Page likes, Page Category, Page Checkin's, Page Talking About.

- 5 esenciales primarias que representan cantidad de comentarios para distintos tiempos en relacion al base date/time.  La cantidad de comentarios se hace *para atras*, es decir una vez que defino el base date time, cuento para atras cuantos comentarios hubo. en distintos intervalos

C1. Cantidad total de comentarios antes del basedatetime seleccionado. Se define 72hs no se porque. 
C2. cantidad de comentarios 24 hs antes del selected BDT.

# Importar librerías

Importar aquellas librerías que serán utilizadas en el trabajo.

In [2]:
import pandas as pd
import time
import itertools
import random
import numpy as np


# Cargar datos
Cargar los datos de entrenamiento.

In [19]:
X_var1 = pd.read_csv("Dataset/Training/Features_Variant_1.csv")
X_var2 = pd.read_csv("Dataset/Training/Features_Variant_2.csv")
X_var3 = pd.read_csv("Dataset/Training/Features_Variant_3.csv")
X_var4 = pd.read_csv("Dataset/Training/Features_Variant_4.csv")
X_var5 = pd.read_csv("Dataset/Training/Features_Variant_5.csv")

X_var1.columns = ['Page Popularity/likes', 'Page Checkins', 'Page talking about ', 'Page Category ', 'Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV', 'CMNTS C1','CMNTS C2','CMNTS C3','CMNTS C4','CMNTS C5','BASE TIME', 'Post length', 'Post Share Count','Post Promotion Status ',' hrs for which we have the target variable/ comments received.  ','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','TARGET VALUE IN NEYT H HOURS']
X_var2.columns = ['Page Popularity/likes', 'Page Checkins', 'Page talking about ', 'Page Category ', 'Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV', 'CMNTS C1','CMNTS C2','CMNTS C3','CMNTS C4','CMNTS C5','BASE TIME', 'Post length', 'Post Share Count','Post Promotion Status ',' hrs for which we have the target variable/ comments received.  ','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','TARGET VALUE IN NEXT H HOURS']
X_var3.columns = ['Page Popularity/likes', 'Page Checkins', 'Page talking about ', 'Page Category ', 'Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV', 'CMNTS C1','CMNTS C2','CMNTS C3','CMNTS C4','CMNTS C5','BASE TIME', 'Post length', 'Post Share Count','Post Promotion Status ',' hrs for which we have the target variable/ comments received.  ','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','TARGET VALUE IN NEXT H HOURS']
X_var4.columns = ['Page Popularity/likes', 'Page Checkins', 'Page talking about ', 'Page Category ', 'Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV', 'CMNTS C1','CMNTS C2','CMNTS C3','CMNTS C4','CMNTS C5','BASE TIME', 'Post length', 'Post Share Count','Post Promotion Status ',' hrs for which we have the target variable/ comments received.  ','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','TARGET VALUE IN NEXT H HOURS']
X_var5.columns = ['Page Popularity/likes', 'Page Checkins', 'Page talking about ', 'Page Category ', 'Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV','Essential DRV', 'CMNTS C1','CMNTS C2','CMNTS C3','CMNTS C4','CMNTS C5','BASE TIME', 'Post length', 'Post Share Count','Post Promotion Status ',' hrs for which we have the target variable/ comments received.  ','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','PUBlished Weekdays (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','BDT weekday (bin)','TARGET VALUE IN NEXT H HOURS']

print(X_var1.shape,X_var2.shape,X_var3.shape,X_var4.shape,X_var5.shape )

#autor dice que esta todo ok, pero voy a chequear si hay NaNs
print(X_var1.isnull().values.any(),X_var2.isnull().values.any(),X_var3.isnull().values.any(),X_var4.isnull().values.any(),X_var5.isnull().values.any(), )

# Ningun data set tiene Nans. todos los datasets tienen misma cantidad de features. Digo que es tidy. 



(40948, 54) (81311, 54) (121097, 54) (160423, 54) (199029, 54)
False False False False False


In [24]:
Y_test_set = pd.read_csv("Dataset/Testing/Features_TestSet.csv")
Y_case1

print(Y_test_set.shape)

(10043, 54)


# Análisis exploratorio básico y preprocesamiento de los datos

Análisis de los datos para conocer los mismos, ver datos faltantes, decidir cómo tratarlos, ver distribuciones, relaciones, etc. Procesar los datos centrándolos, reescalando, codificando, reduciendo dimensiones, etc. según considere necesario. 

# Modelos y evaluación
Probar diferentes modelos para predecir la variable objetivo. Calcular las métricas que considere relevantes. Comentar los resultados. 